# Рубежный контроль №2
Вариант **№4**

Студент: **Кучеренко М.А.**  
Группа: **ИУ5-21М**

### Классификатор №1
LogisticRegression

### Классификатор №2
Multinomial Naive Bayes (MNB)

Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.
Необходимо сформировать два варианта векторизации признаков - на основе `CountVectorizer` и на основе `TfidfVectorizer`.
Для каждого метода необходимо оценить качество классификации. Сделайте вывод о том, какой вариант векторизации признаков в паре с каким классификатором показал лучшее качество.

Набор данных - [20 newsgroups text dataset](https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset)

Классы: 20
Выборка: 18846

In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  

In [2]:
categories = ["sci.crypt", "sci.electronics", "talk.religion.misc", "rec.sport.baseball"]
newsgroups = fetch_20newsgroups(subset='train', categories=categories)
data = newsgroups['data']

In [3]:
def accuracy_score_for_classes(y_true, y_pred):
    df = pd.DataFrame(data={'t': y_true, 'p': y_pred})
    res = dict()
    for c in np.unique(y_true):
        temp_data_flt = df[df['t'] == c]
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values
        )
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(y_true, y_pred):
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs) > 0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [4]:
# С помощью CountVectorizer преобразуем коллекцию текстовых данных в матрицу счётчиков токенов
vocabVect = CountVectorizer()
vocabVect.fit(data)
corpusVocab = vocabVect.vocabulary_
print(f'Feature count - {len(corpusVocab)}')

Feature count - 33282


In [13]:
first_el = 0
last_el = 9

for word in list(corpusVocab)[first_el:last_el]:
    print(f'{word:10}: {corpusVocab[word]}')

from      : 14539
philly    : 23632
ravel     : 25268
udel      : 30929
edu       : 12456
robert    : 26356
hite      : 16186
subject   : 29047
re        : 25308


In [17]:
test_features = vocabVect.transform(data)
test_features.todense().shape

(2160, 33282)

In [18]:
test_features.todense()

matrix([[0, 3, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [25]:
# Cross-validation classification
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    max_acc = 0
    
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([('vectorizer', v), ('classifier', c)])
            score = cross_val_score(
                pipeline1,
                newsgroups['data'],
                newsgroups['target'],
                scoring='accuracy',
                cv=3
            ).mean()
            if score > max_acc:
                max_acc = score
                max_v = v
                max_c = c
            print(f'Векторизация:\t {v}\nКлассификатор:\t {c}\nAccuracy:\t {score}')
            print('='*80)
            
    print(f'\nЛучший результат: {max_acc}, {type(max_v).__name__}, {type(max_c).__name__}')

In [26]:
vectorizers_list = (
    CountVectorizer(vocabulary = corpusVocab),
    TfidfVectorizer(vocabulary = corpusVocab),
)
classifiers_list = (
    LogisticRegression(),
    MultinomialNB(),
)

VectorizeAndClassify(vectorizers_list, classifiers_list)

/Users/snipghost/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/snipghost/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Векторизация:	 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None,
                vocabulary={'00': 0, '000': 1, '00000000': 2, '00000000b...
                            '00000001': 4, '00000001b': 5, '00000010': 6,
                            '00000010b': 7, '00000011': 8, '00000011b': 9,
                            '00000100': 10, '00000100b': 11, '00000101': 12,
                            '00000101b': 13, '00000110': 14, '00000110b': 15,
                            '00000111': 16, '00000111b': 17, '00001000': 18,
                            '00001000b': 19, '00001001': 20, '00001001b': 21,
                            '00001010': 22, '0000101

/Users/snipghost/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/snipghost/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Векторизация:	 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use...
                            '00000001': 4, '00000001b': 5, '00000010': 6,
                            '00000010b': 7, '00000011': 8, '00000011b': 9,
                            '00000100': 10, '00000100b': 11, '00000101': 12,
                            '00000101b': 13, '00000110': 14, '00000110b': 15,
                            '00000111': 16, '00000111b': 17, '00001000': 18,
                            '00001000b': 19, '00001001': 20, '00001001b': 21,
                            '00001010': 22, '00001010b'